In [52]:
import pandas as pd
import numpy as np
import igraph as ig
import time
import itertools
import requests
import random
pd.set_option("mode.copy_on_write", True)

In [31]:
USE_PANDAS_PERF=True # if multiple CPU cores are available

In [23]:
def _get_direct_edges_df(
  fids,
  df,
  max_neighbors,
):
    # WARNING we are operating on a shared dataframe...
    # ...inplace=False by default, explicitly setting here for emphasis
    out_df = df[df['i'].isin(fids)].sort_values(by=['v'], ascending=False, inplace=False)[:max_neighbors]
    return out_df

In [27]:
def find_vertex_idx(ig, fid):
    try:
        return ig.vs.find(name=fid).index
    except:
        return None

def _fetch_korder_neighbors(
  fids,graph,max_degree,max_neighbors,min_degree = 1
):

    # vids = [find_vertex_idx(graph.graph, fid) for fid in fids]
    # vids = list(filter(None, vids)) # WARNING - this filters vertex id 0 also
    vids = [vid for fid in fids for vid in [find_vertex_idx(graph, fid)] if vid is not None ]
    if len(vids) <= 0:
        raise HTTPException(status_code=404, detail="Invalid fids")
    try:
        klists = []
        mindist_and_order = min_degree
        limit = max_neighbors
        while mindist_and_order <= max_degree:
            neighbors = graph.neighborhood(
                vids, order=mindist_and_order, mode="out", mindist=mindist_and_order
            )
            # TODO prune the graph after sorting by edge weight
            klists.append(graph.vs[neighbors[0][:limit]]["name"])
            limit = limit - len(neighbors[0])
            if limit <= 0:
                break # we have reached limit of neighbors
            mindist_and_order += 1
        # end of while
        return set(itertools.chain(*klists))
    except ValueError:
        raise HTTPException(status_code=404, detail="Neighbors not found")

In [33]:
def _get_neighbors_edges(
  fids, df, graph, max_degree, max_neighbors,
):

    start_time = time.perf_counter()
    neighbors_df = _get_direct_edges_df(fids, df, max_neighbors)
    print(f"direct_edges_df took {time.perf_counter() - start_time} secs for {len(neighbors_df)} first degree edges")
    max_neighbors = max_neighbors - len(neighbors_df)
    if max_neighbors > 0 and max_degree > 1:

        start_time = time.perf_counter()
        k_neighbors_list = _fetch_korder_neighbors(fids, graph, max_degree, max_neighbors, min_degree=2)
        print(f"{time.perf_counter() - start_time} secs for {len(k_neighbors_list)} neighbors")

        start_time  = time.perf_counter()
        if USE_PANDAS_PERF:
            # if multiple CPU cores are available
            k_df = df.query('i in @k_neighbors_list').query('j in @k_neighbors_list')
        else:
            # filter with an '&' is slower because of the size of the dataframe
            # split the filtering so that indexes can be used if present
            # k_df = graph.df[graph.df['i'].isin(k_neighbors_list) & graph.df['j'].isin(k_neighbors_list)]
            k_df = df[df['i'].isin(k_neighbors_list)]
            k_df = k_df[k_df['j'].isin(k_neighbors_list)]
        # .loc will throw KeyError when fids have no outgoing actions
        ### in other words, some neighbor fids may not be present in 'i'
        # k_df = graph.df.loc[(k_neighbors_list, k_neighbors_list)]
        print(f"k_df took {time.perf_counter() - start_time} secs for {len(k_df)} edges")

        start_time  = time.perf_counter()
        neighbors_df = pd.concat([neighbors_df, k_df])
        print(f"neighbors_df concat took {time.perf_counter() - start_time} secs for {len(neighbors_df)} edges")

    return neighbors_df

In [40]:
def get_neighbors_list(
  fids, df, graph, max_degree = 2, max_neighbors = 100,
):
    df = _get_neighbors_edges(fids, df, graph, max_degree, max_neighbors)
    # WARNING we are operating on a shared dataframe...
    # ...inplace=False by default, explicitly setting here for emphasis
    out_df = df.groupby(by='j')[['v']].sum().sort_values(by=['v'], ascending=False, inplace=False)
    return out_df.index.to_list()

In [45]:
def go_eigentrust(
    pretrust, max_pt_id, localtrust, max_lt_id,
):
    start_time = time.perf_counter()

    lt_len_before = len(localtrust)
    localtrust[:] = (x for x in localtrust if x["i"] != x["j"])
    lt_len_after = len(localtrust)
    if lt_len_before != lt_len_after:
        print(f"dropped {lt_len_before-lt_len_after} records with i == j")

    req = {
        "pretrust": {
            "scheme": "inline",
            # "size": int(max_pt_id)+1, #np.int64 doesn't serialize; cast to int
            "size": max_pt_id,
            "entries": pretrust,
        },
        "localTrust": {
            "scheme": "inline",
            # "size": int(max_lt_id)+1, #np.int64 doesn't serialize; cast to int
            "size": max_lt_id,
            "entries": localtrust,
        },
        "alpha": 0.5,
    }

    response = requests.post(
        f"http://localhost:8080/basic/v1/compute",
        json=req,
        headers={"Accept": "application/json", "Content-Type": "application/json"},
        timeout=3000,
    )

    if response.status_code != 200:
        logger.error(f"Server error: {response.status_code}:{response.reason}")
        raise Exception("Unknown error")
    trustscores = response.json()["entries"]
    print(
        f"eigentrust took {time.perf_counter() - start_time} secs for {len(trustscores)} scores"
    )
    return trustscores

In [48]:
def get_neighbors_scores(
    fids, df, graph, max_degree, max_neighbors
):
    start_time = time.perf_counter()
    df = _get_neighbors_edges(fids, df, graph, max_degree, max_neighbors)
    print(
        f"dataframe took {time.perf_counter() - start_time} secs for {len(df)} edges"
    )

    if df.shape[0] < 1:
        raise Exception("No neighbors")

    stacked = df.loc[:, ("i", "j")].stack()
    pseudo_id, orig_id = stacked.factorize()

    # pseudo_df is a new dataframe to avoid modifying existing shared global df
    pseudo_df = pd.Series(pseudo_id, index=stacked.index).unstack()
    pseudo_df.loc[:, ("v")] = df.loc[:, ("v")]

    if len(fids) > 1:
        # when more than 1 fid in input list, the neighbor edges may not have some input fids.
        pt_fids = orig_id.where(orig_id.isin(fids))
    else:
        pt_fids = fids
    pt_len = len(fids)
    # pretrust = [{'i': fid, 'v': 1/pt_len} for fid in pt_fids]
    pretrust = [
        {"i": orig_id.get_loc(fid), "v": 1 / pt_len}
        for fid in pt_fids
        if not np.isnan(fid)
    ]
    # max_pt_id = max(pt_fids)
    max_pt_id = len(orig_id)

    localtrust = pseudo_df.to_dict(orient="records")
    # max_lt_id = max(df['i'].max(), df['j'].max())
    max_lt_id = len(orig_id)

    print(
        f"max_lt_id:{max_lt_id}, localtrust size:{len(localtrust)},"
        f" max_pt_id:{max_pt_id}, pretrust size:{len(pretrust)}"
    )

    i_scores = go_eigentrust(
        pretrust=pretrust,
        max_pt_id=max_pt_id,
        localtrust=localtrust,
        max_lt_id=max_lt_id,
    )

    # rename i and v to fid and score respectively
    # also, filter out input fids
    fid_scores = [
        {"fid": int(orig_id[score["i"]]), "score": score["v"]}
        for score in i_scores
        if score["i"] not in fids
    ]
    print(
        f"sample fid_scores:{random.sample(fid_scores, min(10, len(fid_scores)))}"
    )
    return fid_scores


In [2]:
lt_df = pd.read_pickle("/tmp/fc_v3engagement_fid_2024-06-18_df.pkl")

In [3]:
lt_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51374922 entries, 0 to 51374921
Data columns (total 3 columns):
 #   Column  Dtype
---  ------  -----
 0   i       int64
 1   j       int64
 2   v       int64
dtypes: int64(3)
memory usage: 1.1 GB


In [5]:
lt_df.head()

,i,j,v
0,388401,313582,206
1,498555,329174,43
2,566245,5650,6
3,384428,361798,13
4,689562,281836,6


In [6]:
%%time
g = ig.Graph.Read_Pickle('/tmp/fc_v3engagement_fid_2024-06-18_ig.pkl')

CPU times: user 56 s, sys: 26.7 s, total: 1min 22s
Wall time: 1min 57s


In [10]:
rainbow_fids = pd.read_csv('/tmp/rainbow_fids_17sep2024.csv')

In [11]:
rainbow_fids.head()

,fid
0,189237
1,193836
2,456528
3,350457
4,536429


In [14]:
rainbow_fids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22227 entries, 0 to 22226
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   fid     22227 non-null  int64
dtypes: int64(1)
memory usage: 173.8 KB


In [35]:
n_df = _get_neighbors_edges(fids=[189237], df=lt_df, graph=g, max_degree=5, max_neighbors=1000)

direct_edges_df took 0.13775267499977417 secs for 88 first degree edges
0.10024687200029803 secs for 912 neighbors
k_df took 2.2711264770000525 secs for 118201 edges
neighbors_df concat took 0.0035934649999944668 secs for 118289 edges


In [36]:
n_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 118289 entries, 2227131 to 51298114
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   i       118289 non-null  int64
 1   j       118289 non-null  int64
 2   v       118289 non-null  int64
dtypes: int64(3)
memory usage: 3.6 MB


In [38]:
n_df.head(10)

,i,j,v
2227131,189237,9135,34
178997,189237,9856,31
975843,189237,2802,28
5057250,189237,6196,28
632648,189237,18537,20
11976224,189237,12,20
2420314,189237,3,14
12095286,189237,99,14
1776912,189237,1668,14
14286344,189237,236812,13


In [41]:
n_list = get_neighbors_list(fids=[189237], df=lt_df, graph=g, max_degree=5, max_neighbors=1000)

direct_edges_df took 1.5809404209976492 secs for 88 first degree edges
0.8265469129983103 secs for 912 neighbors
k_df took 5.576144401999045 secs for 118201 edges
neighbors_df concat took 0.009947200000169687 secs for 118289 edges


In [42]:
len(n_list)

990

In [43]:
n_list

[602,
 403619,
 781098,
 234616,
 247143,
 436577,
 539,
 269694,
 408746,
 12626,
 15983,
 422233,
 8447,
 307834,
 617,
 239,
 20909,
 253127,
 6596,
 403090,
 4482,
 565051,
 8152,
 397392,
 281836,
 321969,
 129,
 3621,
 576,
 15850,
 243300,
 366713,
 402888,
 248216,
 4407,
 270138,
 391793,
 4923,
 477292,
 419741,
 3642,
 12239,
 5431,
 758919,
 472,
 1325,
 285462,
 337018,
 11528,
 12142,
 527115,
 315697,
 13874,
 230238,
 16098,
 414955,
 277952,
 4163,
 528,
 274908,
 446697,
 1689,
 522467,
 190218,
 508409,
 510364,
 475121,
 1606,
 473,
 476677,
 327165,
 369,
 315256,
 308284,
 7143,
 4905,
 478830,
 210628,
 420493,
 6806,
 2134,
 1918,
 4167,
 5818,
 276562,
 16148,
 1355,
 2211,
 2745,
 475771,
 435160,
 557,
 1287,
 308045,
 203666,
 374339,
 4294,
 343622,
 12921,
 533,
 194372,
 1110,
 475296,
 4528,
 475142,
 194410,
 471309,
 15513,
 14516,
 516505,
 466688,
 12048,
 451,
 880,
 14364,
 16085,
 440352,
 542351,
 288425,
 310124,
 9391,
 300265,
 375536,
 475275

In [53]:
n_scores = get_neighbors_scores(fids=[189237], df=lt_df, graph=g, max_degree=5, max_neighbors=1000)

direct_edges_df took 0.17843161700147903 secs for 88 first degree edges
0.08317014900239883 secs for 912 neighbors
k_df took 1.8651378099966678 secs for 118201 edges
neighbors_df concat took 0.003010766005900223 secs for 118289 edges
dataframe took 2.1303104929975234 secs for 118289 edges
max_lt_id:1000, localtrust size:118289, max_pt_id:1000, pretrust size:1
eigentrust took 0.6566140060022008 secs for 89 scores
sample fid_scores:[{'fid': 9856, 'score': 0.02316890879755531}, {'fid': 571586, 'score': 0.0007473841547598486}, {'fid': 1493, 'score': 0.0029895366190393945}, {'fid': 4179, 'score': 0.0007473841547598486}, {'fid': 25, 'score': 0.0007473841547598486}, {'fid': 3727, 'score': 0.002242152464279546}, {'fid': 808057, 'score': 0.0029895366190393945}, {'fid': 584364, 'score': 0.0007473841547598486}, {'fid': 773618, 'score': 0.0007473841547598486}, {'fid': 4129, 'score': 0.0007473841547598486}]


In [55]:
n_scores

[{'fid': 189237, 'score': 0.6666666669771075},
 {'fid': 9135, 'score': 0.02541106126183485},
 {'fid': 9856, 'score': 0.02316890879755531},
 {'fid': 2802, 'score': 0.020926756333275762},
 {'fid': 6196, 'score': 0.020926756333275762},
 {'fid': 18537, 'score': 0.014947683095196972},
 {'fid': 12, 'score': 0.014947683095196972},
 {'fid': 3, 'score': 0.010463378166637881},
 {'fid': 99, 'score': 0.010463378166637881},
 {'fid': 1668, 'score': 0.010463378166637881},
 {'fid': 236812, 'score': 0.009715994011878032},
 {'fid': 222804, 'score': 0.008968609857118184},
 {'fid': 241781, 'score': 0.008221225702358333},
 {'fid': 18513, 'score': 0.006726457392838637},
 {'fid': 424038, 'score': 0.006726457392838637},
 {'fid': 680, 'score': 0.006726457392838637},
 {'fid': 19832, 'score': 0.005231689083318941},
 {'fid': 2252, 'score': 0.005231689083318941},
 {'fid': 7806, 'score': 0.005231689083318941},
 {'fid': 2282, 'score': 0.005231689083318941},
 {'fid': 246871, 'score': 0.004484304928559092},
 {'fid': 1